In [1]:
# Set NEWT session id
newt_sessionid = '57311ce617ff392d7b6cb6104b0a18dc'
nersc_repository = None

In [2]:
# Load Gaia and connect to girder using NEWT session id
import geolib
girder = geolib.connect(girder_url='http://turtleland4:8080', newt_sessionid=newt_sessionid)


In [3]:
# Todo select item from ESS-DIVE

# As a surrogate, use file that was uploaded manually
resource_type = 'item'
resource_id = '5c88076206a1b5013e1c6d77'    # /user/admin/Public/local/SFBay_grayscale.tif
girder_url = 'girder://{}/{}'.format(resource_type, resource_id)

essdive_object = geolib.create(girder_url)


In [4]:
# Get bounds
# metadata = essdive_object.get_metadata()
metadata = {
 'bounds': {'coordinates': [[[-122.71879201711467, 37.45219874192699],
    [-122.71879201711467, 38.052231141926995],
    [-122.11875961711466, 38.052231141926995],
    [-122.11875961711466, 37.45219874192699],
    [-122.71879201711467, 37.45219874192699]]],
  'type': 'Polygon'},
 'height': 4323,
 'width': 4323}

# Create polygon to display bounds
bounds = metadata.get('bounds')
coordinates = bounds.get('coordinates')
import geojson
geometry = geojson.Polygon(coordinates)
essdive_feature = geojson.Feature(geometry=geometry, properties={'fillColor': 'black', 'fillOpacity': 0.2})
essdive_object = geolib.create(essdive_feature)

# Generate small crop geometry
bounds  = coordinates
bounds = bounds[0]
x = (bounds[0][0] + bounds[2][0]) / 2.0
y = (bounds[0][1] + bounds[2][1]) / 2.0

# Use small percentage of height & width
dx = 0.12 * (bounds[2][0] - bounds[0][0])
dy = 0.16 * (bounds[2][1] - bounds[0][1])
poly = [[
    [x,y], [x+dx,y+dy], [x-dx,y+dy], [x-dx,y-dy], [x+dx,y-dy]
]]
geometry = geojson.Polygon(poly)
crop_feature = geojson.Feature(geometry=geometry, properties={'fillColor': 'magenta', 'fillOpacity': 0.5})
crop_object = geolib.create(crop_feature)

scene1 = geolib.show([crop_object, essdive_object])
display(scene1)


scene(center={'x': 0.0, 'y': 0.0}, layout=Layout(align_self='stretch', height='400px'))

In [5]:
import ipywidgets as widgets
z = widgets.FloatSlider(min=1, max=16, value=scene1.zoom, layout=dict(width='95%'))
newlink = widgets.jslink((z, 'value'), (scene1, 'zoom'))
display(z)

FloatSlider(value=9.328175, layout=Layout(width='95%'), max=16.0, min=1.0)

In [6]:
# To run on Girder:
# cropped_object = gaia.crop(essdiv_object, crop_object)

# To run on NERSC
import getpass
while not nersc_repository:
    nersc_repository = getpass.getpass('Enter NERSC repository (account): ')
cori_job = geolib.submit_crop(essdive_object, crop_object, nersc_repository)
cori_job

Enter NERSC repository (account):  ·····


(CumulusClient) ginterface <gaia.io.girder_interface.GirderInterface object at 0x7f2b4ffac828>
(CumulusClient) gclient <girder_client.GirderClient object at 0x7f2b4ffac7b8>
set girder_client <girder_client.GirderClient object at 0x7f2b4ffac7b8>
user {'groups': [], 'emailVerified': True, 'status': 'enabled', 'created': '2018-01-18T18:39:26.139000+00:00', 'lastName': 'Tourtellott', '_id': '5a60e9de0640fd01195132e4', 'groupInvites': [], 'email': 'john.tourtellott@kitware.com', '_accessLevel': 2, 'login': 'johnt', 'size': 1125529012, 'public': True, '_modelType': 'user', 'admin': True, 'firstName': 'John'}
private_folder_id 5a60e9de0640fd01195132e6
checking girder_client <girder_client.GirderClient object at 0x7f2b4ffac7b8>
Creating cluster on cori
status created
status running
Creating SLURM script geolib
script_id 5c8ad6ee0640fd012102b898
Creating job geolib
Created job folder 0ecff36a39f1463fb1dc7ae092e56780
Created job_id 5c8ad6ef0640fd012102b89c
Uploading geometry file
Submitting job
